The code implies how the model of Gemini works on the linkedin profiles in arrays
I have run it, but due to lack of paid version of gemini, I was able to run only 2 array elements
It is hallucinating no doubt on that

In [ ]:
import os
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown,display

Gemini_base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
google_api_key = os.getenv("GOOGLE_API_KEY")
gemini = OpenAI(base_url=Gemini_base_url, api_key=google_api_key)

def scrap_profile(url):
    headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36 Edg/143.0.0.0"}
    try:
        response = requests.get(url, headers= headers)
        soup= BeautifulSoup(response.text,'html.parser')

        profile_text = soup.get_text(separator=" ", strip= True)
        return profile_text[:10000]
    except Exception as e:
        return f"Error Scraping:{e}"
    
def review_profile(url):
    raw_data = scrap_profile(url)

    system_prompt = (" You are a sarcastic hiring agent , analyse the profile of the candidate. Provide a company that the candidate would be fit in"
    "Identify: 1. Name of the candidate 2.Top skills 3.Experience 4.About . If no infomration available,dont hallucinate.")

    user_prompt = f"Profile Content: {raw_data}"

    response = gemini.chat.completions.create(
        model="gemini-2.5-flash-lite", messages=[{"role":'system',"content":system_prompt}, {'role':'user','content':user_prompt}], temperature=0
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = review_profile(url)
    display(Markdown(summary))

#Running the code for a couple of urls at once:

links= ['www.linkedin.com/in/barira-ushshaque', 'https://www.linkedin.com/in/aquib-javed-2a630875/','https://www.linkedin.com/in/dibya-swain-29633a75/']

for index in links:
    display_summary(index)



